In [40]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import psycopg2

## Conectar Base de datos y Preparacion de los datos a utilizar

In [41]:
conexion = psycopg2.connect(
        dbname="buap",
        user="postgres",
        password="contrasena",
        host="localhost",
        port="5432"
    )
cursor = conexion.cursor()
datos = {}

In [42]:
# Profesores (con nombre)
cursor.execute("SELECT id_profesor, nombre FROM Profesor LIMIT 10")
profesores = cursor.fetchall()
datos['Id_Profesores'] = [row[0] for row in profesores] # Lista de IDs de profesores
datos['nombres_profesor'] = {row[0]: row[1] for row in profesores} # Diccionario de nombres de profesores


In [43]:
datos['Id_Profesores']

[100002999,
 100004288,
 100012622,
 100014033,
 100020344,
 100054211,
 100065344,
 100104677,
 100104866,
 100124799]

In [44]:
# Materias (con nombre)
cursor.execute("SELECT id_materia, nombre, horas_por_semana, id_tipo_clase FROM Materia LIMIT 10")
materias = cursor.fetchall()
datos['id_Materias'] = [row[0] for row in materias] # Lista de IDs de materias
datos['nombres_materia'] = {row[0]: row[1] for row in materias} # Diccionario de nombres de materias
datos['Horas_Semana_Materia'] = {row[0]: row[2] for row in materias} # Diccionario de horas por semana de cada materia
datos['Tipo_Clase'] = {row[0]: row[3] for row in materias} # Diccionario de tipo de clase de cada materia

In [45]:
# Aulas
cursor.execute("SELECT id_aula, id_edificio, id_tipo_clase FROM Aula LIMIT 10")
rows = cursor.fetchall()

datos['Aulas'] = [(row[0], row[1]) for row in rows] # Lista de IDs compuestos (id_aula, id_edificio)
datos['Tipo_Aula'] = {(row[0], row[1]): row[2] for row in rows}# Diccionario: clave compuesta (id_aula, id_edificio) -> tipo clase


# Bloques horarios
datos['Dia'] = list(range(1, 6)) # Lunes a Viernes
datos['Hora'] = list(range(1, 11)) # 10 bloques horarios por día/ de 8 am a 6 pm

In [46]:
cursor.execute("""
    SELECT A.id_aula, A.id_edificio, M.id_materia,
           CASE WHEN A.id_tipo_clase = M.id_tipo_clase THEN 1 ELSE 0 END AS compatible,
           M.id_tipo_clase AS tipo_materia,
           A.id_tipo_clase AS tipo_aula
    FROM Aula A
    CROSS JOIN Materia M
    WHERE A.id_aula = ANY(%s) AND M.id_materia = ANY(%s)
""", ([a[0] for a in datos['Aulas']], datos['id_Materias']))  # solo ids de aula

datos['Compatibiliadad_Aula_Materia'] = {
    ((row[0], row[1]), row[2]): {
        'compatible': row[3],
        'tipo_materia': row[4],
        'tipo_aula': row[5]
    }
    for row in cursor.fetchall()
}


In [47]:
from collections import defaultdict
datos['Materias_por_Profesor'] = defaultdict(list)
datos['Profesores_por_Materia'] = defaultdict(list)
datos['Profesores_Imaginarios'] = []
datos['Profesores_Totales'] = []

cursor.execute("""
    SELECT id_profesor, id_materia 
    FROM profesor_materia
    WHERE id_profesor = ANY(%s) AND id_materia = ANY(%s)
""", (datos['Id_Profesores'], datos['id_Materias']))

for profe_id, materia_id in cursor.fetchall():
    datos['Materias_por_Profesor'][profe_id].append(materia_id)
    datos['Profesores_por_Materia'][materia_id].append(profe_id)

materias_sin_profesor = [m for m in datos['id_Materias'] if m not in datos['Profesores_por_Materia']]

profesores_imaginarios = [f"PA_{i}" for i in range(len(materias_sin_profesor))]
datos['Profesores_Imaginarios'] = profesores_imaginarios

#Asignación de profesores imaginarios
for i, materia_id in enumerate(materias_sin_profesor):
    profe_imag = profesores_imaginarios[i]
    datos['Profesores_por_Materia'][materia_id].append(profe_imag)
    datos['Materias_por_Profesor'][profe_imag].append(materia_id)

datos['Profesores_Totales'].extend(datos['Id_Profesores'])
datos['Profesores_Totales'].extend(profesores_imaginarios)

datos['nombres_profesor'].update({
    p: f"ProfFicticio_{i}" for i, p in enumerate(profesores_imaginarios)
})


In [48]:
print(type(datos))  # Verifica el tipo de 'datos'
print(datos.keys())  # Muestra las claves del diccionario
print(len(datos))    # Muestra cuántos elementos hay en el diccionario

<class 'dict'>
dict_keys(['Id_Profesores', 'nombres_profesor', 'id_Materias', 'nombres_materia', 'Horas_Semana_Materia', 'Tipo_Clase', 'Aulas', 'Tipo_Aula', 'Dia', 'Hora', 'Compatibiliadad_Aula_Materia', 'Materias_por_Profesor', 'Profesores_por_Materia', 'Profesores_Imaginarios', 'Profesores_Totales'])
15


## Crear Modelo de Pyomo

---

### Conjuntos

- $P: \text{Profesores}$
    - $P_R: \text{Profesores Reales}$
    - $P_I: \text{Profesores Imaginarios}$
    - $P = P_R \cup\ P_I$
- $M: \text{Materias}$
- $A: \text{Aulas}$
    - $A = A_{salon}\ \cup\ A_{edificio}$
- $BH: \text{Bloques Horarios}$
    - $D: \text{Dias}, D = \{1, 2,\ \dots, 5\}$
    - $H: \text{Horarios}, H = \{1, 2,\ \dots, 10\}$
    - $BH = D \times H = \{(d, h)\ |\ d \in D, h \in H\}$

In [50]:
# Definir los conjuntos
model = pyo.ConcreteModel()

# Conjuntos
model.Profesores = pyo.Set(initialize=datos['Profesores_Totales'])
model.Materias = pyo.Set(initialize=datos['id_Materias'])
model.Aulas = pyo.Set(dimen=2, initialize=[(aula_id, edificio_id) for aula_id, edificio_id in datos['Aulas']])
model.BloquesHorarios = pyo.Set(dimen=2, initialize=[(dia, hora) for dia in datos['Dia'] for hora in datos['Hora']])
model.Dias = pyo.Set(initialize=datos['Dia'])

### Parametros

TODO: AGREGAR FUNCION PARA DETECTAR PROF. IMAGINARIOS
- $\text{HorasSemanaMateria}(m) = n$, donde $n$ es el numero de horas que la materia $m$ tiene por semana
- $\text{TipoClase}(m) = t$, donde $t \in \{1,2,3,4\}$ es el tipo de clase de la materia $m$
- $\text{TipoAula}(a) = t$, donde $t \in \{1,2,3,4\}$ es el tipo de aula de la aula $a$
- $ \text{Compatibles}(a, m) \begin{cases} 1 & \text{si el aula}\ a\ \text{es compatible con la materia}\ m\\ 0 & \text{si no} \end{cases}$

- $ \text{Relacion}(p, m) \begin{cases} 1 & \text{si el profesor}\ p\ \text{puede dar la materia}\ m\\ 0 & \text{si no} \end{cases}$

In [51]:
# Parámetros

model.Horas_Semana_Materia = pyo.Param(model.Materias, initialize=datos['Horas_Semana_Materia'])
model.Tipo_Clase = pyo.Param(model.Materias, initialize=datos['Tipo_Clase'])
model.Tipo_Aula = pyo.Param(model.Aulas, initialize=datos['Tipo_Aula'])
model.Compatibilidad_Aula_Materia = pyo.Param(model.Aulas, model.Materias, initialize={
    (aula, materia): datos['Compatibiliadad_Aula_Materia'].get((aula, materia), {}).get('compatible', 0)
    for aula in model.Aulas for materia in model.Materias
})  # Compatibilidad entre aulas y materias (0 o 1)
# Relación entre profesores y las materias que pueden enseñar
model.Relacion_Profesor_Materia = pyo.Param(model.Profesores, model.Materias, initialize={
    (profe_id, materia_id): 1 if materia_id in datos['Materias_por_Profesor'][profe_id] else 0
    for profe_id in model.Profesores for materia_id in model.Materias
})



### Variables

- $
x^{i, j}_{k, t} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ tiene asignada la materia } j
    \text{ en el aula } k
    \text{ en el horario } t \\
    0 & \text{si no}
\end{cases}
$

- $
y_{i, j} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ puede enseñar la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
\hat{y}_{i, j} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ tiene asignada la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
z_{k, j} =
\begin{cases}
    1 & \text{si el aula } k
    \text{ esta asignada a la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
\hat{z}_{j, d} =
\begin{cases}
    1 & \text{si la materia } j
    \text{ se imparte el dia } d \\
    0 & \text{si no}
\end{cases}
$

- $
s_{j, t, d} =
\begin{cases}
    1 & \text{si el curso } j
    \text{ comienza a la hora } t
    \text{ en el dia } d \\
    0 & \text{si no}
\end{cases}
$

In [54]:
# Variables

# Crear índice de combinaciones válidas para X (incluyendo profesores imaginarios)
X_index = [
    (p, m, a, bh)
    for p in datos['Profesores_Totales']  # Cambiado a Profesores_Totales (incluye reales e imaginarios)
    for m in datos['id_Materias']
    if m in datos['Materias_por_Profesor'].get(p, [])  # Usar get para evitar KeyError
    for a in datos['Aulas']
    if datos['Compatibiliadad_Aula_Materia'].get((a, m), {}).get('compatible', 0) == 1  # Aulas compatibles
    for bh in [(d, h) for d in datos['Dia'] for h in datos['Hora']]
]

PROFESOR = 0
MATERIA = 1
AULA = 2
EDIFICIO = 3
DIA = 4
HORA = 5

model.X = pyo.Var(X_index, domain=pyo.Binary)

# Variables generales: profesor puede enseñar materia (relación) - incluye imaginarios
Y_index = [(p, m) for p in datos['Profesores_Totales'] for m in datos['Materias_por_Profesor'].get(p, [])]
model.Y = pyo.Var(Y_index, domain=pyo.Binary)

# Variables generales: profesor tiene asignada la materia
Y_hat_index = [(p, m) for p in datos['Profesores_Totales'] for m in datos['id_Materias']]
model.Y_hat = pyo.Var(Y_hat_index, domain=pyo.Binary)

# Variables generales: aula asignada a materia (sin cambios)
Z_index = [(a, m) for (a, m), val in datos['Compatibiliadad_Aula_Materia'].items() if val['compatible'] == 1]
model.Z = pyo.Var(Z_index, domain=pyo.Binary)

# Variables generales: materia impartida en dia d
Z_hat_index = [(m, d) for m in datos['id_Materias'] for d in datos['Dia']]
model.Z_hat = pyo.Var(Z_hat_index, domain=pyo.Binary)

# Variable Materia X es dada en hora H el dia D
S_Index = [(m, h, d) for m in datos['id_Materias'] for h in datos['Hora'] for d in datos['Dia']]
model.S = pyo.Var(S_Index, domain=pyo.Binary)

print(f"Variables reducidas de {len(datos['Profesores_Totales'])*len(datos['id_Materias'])*len(datos['Aulas'])*len(datos['Dia'])*len(datos['Hora'])} a {len(X_index)}")

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.

### Restricciones

- El curso $j$ debe tener asignado un profesor:
$$
\sum_{i \in P} \hat{y}_{i, j} = 1,\ \forall\ j \in M
$$

In [56]:
if hasattr(model, 'Restriccion_Materia_Tiene_Profe'):
    model.del_component(model.Restriccion_Materia_Tiene_Profe)

def materia_tiene_profe_rule(model, m):
    return sum(
        model.Y_hat[p, m]
        for p in model.Profesores
    ) == 1

model.Restriccion_Materia_Tiene_Profe = pyo.Constraint(model.Materias, rule=materia_tiene_profe_rule)

- El profesor $i$ debe impartir todas las horas del curso $j$, si el es seleccionada para impartirlo:
$$
\sum_{k \in A} \sum_{t \in BH} x^{i, j}_{k, t} = \text{HorasSemanaMateria}(j) \cdot \hat{y}_{i, j} \quad \forall\ j \in M, i \in P
$$

In [57]:
if hasattr(model, 'Restriccion_Horas_Semanales'):
    model.del_component(model.Restriccion_Horas_Semanales)

# Asignar exactamente las horas semanales requeridas por cada materia
def horas_semana_materia_rule(model, p, m):
    return sum(
        model.X[idx]
        for idx in model.X
        if idx[MATERIA] == m
        and idx[PROFESOR] == p
    ) == datos['Horas_Semana_Materia'][m] * model.Y_hat[p, m]

model.Restriccion_Horas_Semanales = pyo.Constraint(datos['Id_Profesores'],
                                                   datos['id_Materias'],
                                                   rule=horas_semana_materia_rule)

TODO: ESCRIBIR RESTRICCION VARIABLE S

PENDIENTE CHECAR
- Variable de Materia Dia Activa: 
$$
\beta_{j, d} =
\begin{cases}
    1 & \text{si la materia } j
    \text{ tiene clase en el dia } d \\
    0 & \text{si no}
\end{cases}
$$

$$
\sum_{i \in P} \sum_{k \in A} x^{i, j}_{k, t} \ge \beta_{j, d} \quad \forall\ j \in M, d \in D, t \in \{(t,d)\ |\ t \in T\}
$$

In [58]:
# # Variable auxiliar que indica si la materia m tiene clase en el día d
# model.Materia_Dia_Activa = pyo.Var(model.Materias, model.Dias, domain=pyo.Binary)

# # Vincular Materia_Dia_Activa con X: si hay al menos una clase de la materia en el día d, debe ser 1
# def materia_dia_activa_rule(model, m, d):
#     # Suma de todas las X asignadas a la materia m en día d
#     return sum(
#         model.X[idx]
#         for idx in model.X
#         if idx[1] == m and idx[4] == d
#     ) >= model.Materia_Dia_Activa[m, d]
# model.Materia_Dia_Activa_Constraint = pyo.Constraint(model.Materias, model.Dias, rule=materia_dia_activa_rule)

# Forzar mínimo días con clases por materia
# MIN_DIAS_POR_MATERIA = 5
# def min_dias_por_materia_rule(model, m):
#     return sum(model.Materia_Dia_Activa[m, d] for d in model.Dias) >= MIN_DIAS_POR_MATERIA

# model.Min_Dias_Por_Materia_Constraint = pyo.Constraint(model.Materias, rule=min_dias_por_materia_rule)

In [59]:
from collections import defaultdict

# Agrupar las variables X válidas por materia y día-hora usando X_index
bloques_por_materia_dia = defaultdict(lambda: defaultdict(list))

for (p, m, a, (d, h)) in X_index:
    if datos['Horas_Semana_Materia'][m] > 3:
        bloques_por_materia_dia[m][d].append(((p, m, a, (d, h)), h))  # guardamos también la hora

# Crear las restricciones de ventanas deslizantes
model.restricciones_horas_consecutivas = pyo.ConstraintList()

for m in bloques_por_materia_dia:
    for d in bloques_por_materia_dia[m]:
        # Agrupar todas las variables válidas por hora
        hora_dict = defaultdict(list)
        for idx, h in bloques_por_materia_dia[m][d]:
            hora_dict[h].append(model.X[idx])

        # Para cada ventana de 3 bloques consecutivos (1–3, 2–4, ..., 8–10)
        for h_inicio in range(1, 9):  # de 1 a 8
            ventana_vars = []
            for offset in range(3):
                h_actual = h_inicio + offset
                ventana_vars.extend(hora_dict.get(h_actual, []))
            
            if ventana_vars:
                model.restricciones_horas_consecutivas.add(sum(ventana_vars) <= 2)

In [60]:
from collections import defaultdict

X_por_profesor = defaultdict(list)
for idx in X_index:
    p = idx[0]
    X_por_profesor[p].append(idx)

datos['X_por_profesor'] = X_por_profesor

def max_horas_profesor_rule(model, p):
    if p not in X_por_profesor:
        return pyo.Constraint.Skip
    return sum(model.X[idx] for idx in X_por_profesor[p]) <= 20

model.Restriccion_Max_Horas_Profesor = pyo.Constraint(
    model.Profesores, rule=max_horas_profesor_rule
)


- No Solapamiento Profesor (No asignar un profesor a más de una materia en un bloque horario):
$$
\sum_{j \in M} \sum_{k \in A} x^{i, j}_{k, t} \le 1 \quad \forall\ i \in P,\ t \in BH

In [61]:
if hasattr(model, 'Restriccion_Profesor_Unico'):
    model.del_component(model.Restriccion_Profesor_Unico)

def no_solapamiento_profesor_rule(model, p, d, h):
    exprs = [
        model.X[idx]
        for idx in model.X
        if idx[0] == p and idx[4] == d and idx[5] == h
    ]
    
    if not exprs:  # No hay clases asignables en este bloque
        return pyo.Constraint.Skip
    
    return sum(exprs) <= 1

model.Restriccion_Profesor_Unico = pyo.Constraint(
    model.Profesores, model.Dias, datos['Hora'],
    rule=no_solapamiento_profesor_rule
)

- No Solapamiento Aula (No asignar un aula a más de una materia en un bloque horario):
$$
\sum_{i \in P} \sum_{j \in M} x^{i, j}_{k, t} \le 1 \quad \forall\ k \in A,\ t \in BH
$$

In [62]:
if hasattr(model, 'Restriccion_Aula_Unica'):
    model.del_component(model.Restriccion_Aula_Unica)

# No asignar un aula a más de una materia en un bloque horario
def no_solapamiento_aula_rule(model, aula_id, edificio_id, dia, hora):
    aula_edificio = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    
    vars_to_sum = [
        model.X[p, m, aula_edificio, bloque_horario]
        for p in model.Profesores
        for m in model.Materias
        if (p, m, aula_edificio, bloque_horario) in model.X
    ]
    
    if not vars_to_sum:
        return pyo.Constraint.Skip  
    
    return sum(vars_to_sum) <= 1

model.Restriccion_Aula_Unica = pyo.Constraint(
    model.Aulas, model.BloquesHorarios,
    rule=no_solapamiento_aula_rule
)

- Profesor Puede Enseñar (Asignar un profesor a una materia solo si el profesor puede enseñar esa materia):
$$
x^{i, j}_{k, t} \le \text{Relacion}(i,j) \quad \forall\ i \in P,\ j \in M,\ k \in A,\ t \in BH
$$

In [63]:
if hasattr(model, 'Restriccion_Profesor_Compatible'):
    model.del_component(model.Restriccion_Profesor_Compatible)

# Asignar un profesor a una materia solo si el profesor puede enseñar esa materia
def profesor_puede_ensenar_rule(model, p, m, id_aula, id_edificio, dia, hora):
    aula = (id_aula, id_edificio)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Relacion_Profesor_Materia[p, m]


model.Restriccion_Profesor_Compatible = pyo.Constraint(
    X_index,
    rule=profesor_puede_ensenar_rule
)

- Aula Compatible Materia (Asignar un aula a una materia solo si el tipo de aula es compatible con el tipo de materia):

$$
x^{i, j}_{k, t} \le \text{Compatibles}(j,k) \quad \forall\ i \in P,\ j \in M,\ k \in A,\ t \in BH
$$

In [64]:
if hasattr(model, 'Restriccion_Aula_Compatible'):
    model.del_component(model.Restriccion_Aula_Compatible)

# Asignar un aula a una materia solo si el tipo de aula es compatible con el tipo de materia
def aula_compatible_materia_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Compatibilidad_Aula_Materia[aula, m]


model.Restriccion_Aula_Compatible = pyo.Constraint(
    X_index,
    rule=aula_compatible_materia_rule
)


- Si hay una asignacion para una clase a una hora de un maestro, que la materia este

$$
x^{i,j}_{k,t} \le z_{j,k} \quad \forall i \in P, j \in M, k \in A, t \in BH
$$

In [65]:
# %% Restricciones que vinculan variables combinadas con variables generales

if hasattr(model, 'Relacion_X_Z'):
    model.del_component(model.Relacion_X_Z)

def relacion_X_Z_rule(model, p, m, aula_id, edificio_id, dia, hora):
    aula = (aula_id, edificio_id)
    bloque_horario = (dia, hora)
    return model.X[p, m, aula, bloque_horario] <= model.Z[aula, m]

model.Relacion_X_Z = pyo.Constraint(
    X_index,
    rule=relacion_X_Z_rule
)


# 1
# 2 x
# 3 x
# 4

- Solo haya una sesion por dia para una materia $j$ en un dia $d$
$$
\sum_{t \in T} s_{j, t, d} \le \hat{z}_{j, d} \quad j \in M,\ d \in D
$$

$$
\text{para } s \in S
$$

In [65]:
if hasattr(model, 'Restriccion_Sesion_Diaria'):
    model.del_component(model.Restriccion_Sesion_Diaria)

def una_sesion_por_dia_rule(model, m, d):
    return sum(
        model.S[ids]
        for ids in model.S
        if ids[0] == m
        and ids[2] == d
    ) <= model.Z_hat[m, d]

model.Restriccion_Sesion_Diaria = pyo.Constraint(datos['id_Materias'], datos['Dia'], rule=una_sesion_por_dia_rule)

In [66]:
# Restricciones adicionales para asegurar que cada materia tenga asignado al menos un profesor y un aula

if hasattr(model, 'Materia_Tiene_Profesor'):
    model.del_component(model.Materia_Tiene_Profesor)

# Crear diccionario que mapea cada materia a la lista de profesores que la pueden enseñar
profesores_por_materia = {m: [] for m in model.Materias}
for (p, m) in model.Y.index_set():
    profesores_por_materia[m].append(p)

# Ahora la restricción
def materia_tiene_profesor_rule(model, m):
    return sum(model.Y[p, m] for p in profesores_por_materia[m]) >= 1

model.Materia_Tiene_Profesor = pyo.Constraint(model.Materias, rule=materia_tiene_profesor_rule)

In [67]:
def materia_tiene_aula_rule(model, m):
    # Filtra solo índices con materia m
    relevant_indices = [idx for idx in model.Z.index_set() if idx[-1] == m]
    return sum(model.Z[idx] for idx in relevant_indices) >= 1


if hasattr(model, 'Materia_Tiene_Aula'):
    model.del_component(model.Materia_Tiene_Aula)

model.Materia_Tiene_Aula = pyo.Constraint(model.Materias, rule=materia_tiene_aula_rule)


In [68]:
# =============================================
# FUNCIÓN OBJETIVO ACTUALIZADA
# =============================================

# Parámetros de ponderación
peso_profesores_activos = 1.0      # Minimizar uso de profesores reales activos
peso_aulas = 1.0                   # Minimizar cantidad de aulas usadas
peso_asignaciones = -0.1           # Maximizar densidad horaria (signo negativo)
peso_distribucion = -0.05          # Maximizar distribución uniforme (signo negativo)
M = 1000                           # Alta penalización por usar profesores imaginarios

def objetivo_actualizado(model):
    # 1. Profesores reales activos utilizados (minimizar)
    profesores_activos_utilizados = sum(
        model.Y[p, m] 
        for p in datos['Id_Profesores']  # Cambiado a Id_Profesores
        for m in model.Materias 
        if (p, m) in model.Y
    )
    
    # 2. Aulas utilizadas (minimizar)
    aulas_utilizadas = sum(
        model.Z[a, m] 
        for a in model.Aulas 
        for m in model.Materias 
        if (a, m) in model.Z
    )
    
    # 3. Total de asignaciones (maximizar)
    asignaciones_totales = sum(
        model.X[idx] 
        for idx in model.X
    )
    
    # 4. Distribución horaria (maximizar uniformidad)
    distribucion = 0
    for p in model.Profesores:
        # Solo considerar profesores reales activos
        if p in datos['Id_Profesores']:
            horas_por_dia = {d: 0 for d in model.Dias}
            for d in model.Dias:
                horas_por_dia[d] = sum(
                    model.X[p, m, a, (d, h)] 
                    for m in model.Materias 
                    for a in model.Aulas 
                    for h in datos['Hora']
                    if (p, m, a, (d, h)) in model.X
                )
            media = sum(horas_por_dia.values()) / max(1, len(model.Dias))
            varianza = sum((h - media)**2 for h in horas_por_dia.values()) / max(1, len(model.Dias))
            distribucion += varianza
    
    # 5. Penalización por profesores imaginarios (minimizar fuertemente)
    profesores_imaginarios_utilizados = sum(
        model.Y[p, m] 
        for p in datos['Profesores_Imaginarios']
        for m in model.Materias 
        if (p, m) in model.Y
    )
    
    return (
        peso_profesores_activos * profesores_activos_utilizados +
        peso_aulas * aulas_utilizadas +
        peso_asignaciones * asignaciones_totales +
        peso_distribucion * distribucion +
        M * profesores_imaginarios_utilizados
    )

# Eliminar objetivo anterior si existe
if hasattr(model, 'Objetivo'):
    model.del_component(model.Objetivo)

model.Objetivo = pyo.Objective(
    rule=objetivo_actualizado, 
    sense=pyo.minimize
)

In [69]:
def resolver_modelo(model, mipgap=0.02, threads=4, emphasis=1):
    from pyomo.opt import SolverFactory, TerminationCondition

    solver = SolverFactory('cplex')

    if not solver.available():
        raise RuntimeError("CPLEX no está disponible. Verifica la instalación o el PATH.")

    resultado = solver.solve(model, tee=True)

    # Verificar si la solución fue óptima o factible
    condicion = resultado.solver.termination_condition
    if condicion in [TerminationCondition.optimal, TerminationCondition.feasible]:
        print(f"Solución encontrada: {condicion}")
    else:
        print(f"Problema durante la resolución: {condicion}")
        print("Revisa el modelo o ajusta parámetros del solver.")

    return resultado


In [70]:
def mostrar_bloque(dia, hora):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    dia_str = dias[dia - 1]  # porque 1 = Lunes
    hora_inicio = 7 + hora  # si hora=1 corresponde a 8am
    hora_fin = hora_inicio + 1
    return f"{dia_str} {hora_inicio:02d}:00 - {hora_fin:02d}:00"

def mostrar_asignacion(model):
    print("\nAsignaciones de profesores, materias, aulas y horarios:")
    for idx in model.X:
        if pyo.value(model.X[idx]) > 0.5:
            # idx = (p, m, aula, edificio, dia, hora)
            p, m, aula, edificio, dia, hora = idx
            print(f"Profesor: {datos['nombres_profesor'].get(p, p)} | "
                  f"Materia: {datos['nombres_materia'].get(m, m)} | "
                  f"Aula: {aula} (Edificio {edificio}) | "
                  f"Horario: {mostrar_bloque(dia, hora)}")



In [71]:
# Resolver el modelo
resultados = resolver_modelo(model)

# Verificar si encontró solución
if (resultados.solver.status == pyo.SolverStatus.ok and
    resultados.solver.termination_condition == TerminationCondition.optimal):

    print("Solución óptima encontrada.\n")
    mostrar_asignacion(model)

else:
    print("No se encontró solución óptima.")
    print("Estado:", resultados.solver.status)
    print("Condición de terminación:", resultados.solver.termination_condition)



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\bdgae\AppData\Local\Temp\tmpiazz04k2.cplex.log' open.
CPLEX> Problem 'C:\Users\bdgae\AppData\Local\Temp\tmp1t0ofr2q.pyomo.lp' read.
Read time = 0.11 sec. (0.65 ticks)
CPLEX> Problem name         : C:\Users\bdgae\AppData\Local\Temp\tmp1t0ofr2q.pyomo.lp
Objective sense      : Minimize
Variables            :    2622  [Binary: 2622]
Objective nonzeros   :    2412
Linear constraints   :    7979  [Less: 7949,  Greater: 20,  Equal: 10]
  Nonzeros           :   20382
  RHS nonzeros       :    5579

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   : 